<a href="https://colab.research.google.com/github/namozhdehi/Pneumonia/blob/main/05_modeling_Pneumonia_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 Modeling<a id='5_Modeling'></a>

## 5.1 Contents<a id='5.1_Contents'></a>
* [5 Modeling](#5_Modeling)
  * [5.1 Contents](#5.1_Contents)
  * [5.2 Introduction](#5.2_Introduction)
  * [5.3 Imports](#5.3_Imports)
  * [5.4 Loading Data](#5.4_Loading_Data)
  * [5.5 Model Architecture](#5.5_Model_Architecture)
  * [5.6 Training the Model](#5.6_Training)
  * [5.7 Validation and Testing](#5.7_Validation_Testing)
  * [5.8 Save the Model](#5.8_Save_Model)
  * [5.9 Summary](#5.9_Summary)

## 5.2 Introduction<a id='5.2_Introduction'></a>

This notebook focuses on improving and training a deep learning model using TensorFlow and Keras for pneumonia detection based on chest X-ray images. We use a pre-trained EfficientNet-B7 model for binary classification (Pneumonia vs Normal). The dataset is preprocessed and augmented with advanced techniques, and the model's performance is evaluated using accuracy, loss, and other evaluation metrics.

## 5.3 Imports<a id='5.3_Imports'></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
import os

**Enable Mixed Precision and XLA Compilation**

In [2]:
# Enable XLA and Mixed Precision
from tensorflow.keras import mixed_precision
tf.config.optimizer.set_jit(True)  # Enable XLA for speed

# Enable mixed precision training
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print(f"Compute dtype: {policy.compute_dtype}")
print(f"Variable dtype: {policy.variable_dtype}")

Compute dtype: float16
Variable dtype: float32


## 5.4 Loading Data<a id='5.4_Loading_Data'></a>

In [3]:
# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 128  # Increase batch size for faster processing

# Directories for training, validation, and test data
train_dir = 'Data/chest_xray/train'
val_dir = 'Data/chest_xray/val'
test_dir = 'Data/chest_xray/test'

# Use tf.data.Dataset instead of ImageDataGenerator for performance
def process_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # Normalize
    return image, label

# Function to load and preprocess images
def load_data(directory, batch_size):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        label_mode="binary",
        image_size=IMG_SIZE,
        batch_size=batch_size
    )
    dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_ds = load_data(train_dir, BATCH_SIZE)
val_ds = load_data(val_dir, BATCH_SIZE)
test_ds = load_data(test_dir, BATCH_SIZE)

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


## 5.4.1 Setup Kaggle API

This code sets up the Kaggle API credentials to download the "chest-xray-pneumonia" dataset from Kaggle, unzips the dataset into a folder named "Data/chest_xray," and checks if the directory exists, raising an error if it doesn't.

In [4]:
'''
# Set up Kaggle API credentials
os.environ['KAGGLE_CONFIG_DIR'] = "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/kaggle.json"  # Update this path

# Download the dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Unzip the downloaded file
import zipfile

with zipfile.ZipFile('chest-xray-pneumonia.zip', 'r') as zip_ref:
    zip_ref.extractall('Data')  # Extract to a folder named 'chest_xray'

# Define the data directory where the dataset is extracted
data_dir = 'Data/chest_xray'


# Check if data_dir exists
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"The dataset directory '{data_dir}' does not exist. Please check the path.")
    '''

'\n# Set up Kaggle API credentials\nos.environ[\'KAGGLE_CONFIG_DIR\'] = "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/kaggle.json"  # Update this path\n\n# Download the dataset\n!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia\n\n# Unzip the downloaded file\nimport zipfile\n\nwith zipfile.ZipFile(\'chest-xray-pneumonia.zip\', \'r\') as zip_ref:\n    zip_ref.extractall(\'Data\')  # Extract to a folder named \'chest_xray\'\n\n# Define the data directory where the dataset is extracted\ndata_dir = \'Data/chest_xray\'\n\n\n# Check if data_dir exists\nif not os.path.exists(data_dir):\n    raise FileNotFoundError(f"The dataset directory \'{data_dir}\' does not exist. Please check the path.")\n    '

## 5.5 Model Architecture<a id='5.5_Model_Architecture'></a>

In [5]:
# Multi-GPU support using MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Load EfficientNetB7 model pre-trained on ImageNet
    base_model = EfficientNetB7(include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

    # Freeze base layers to prevent updating during initial training
    base_model.trainable = False

    # Add custom layers for binary classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Reduce dropout for faster convergence
    output = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

    # Create the full model
    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

## 5.6 Training the Model<a id='5.6_Training'></a>

In [ ]:
# Callbacks for early stopping and learning rate reduction
early_stop = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)

with strategy.scope():
    # Train the model using mixed precision and XLA optimizations
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=6,  # Reduce epochs for quicker training
        callbacks=[early_stop, reduce_lr]
    )

    # Unfreeze base layers and fine-tune the model
    base_model.trainable = True
    model.compile(optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Fine-tune the model
    history_finetune = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=3,  # Further reduce fine-tuning epochs
        callbacks=[early_stop, reduce_lr]
    )


Epoch 1/6


## 5.7 Plot Training History<a id='5.7_History'></a>

In [ ]:
# Plot training & validation accuracy and loss

def plot_history(history, history_finetune):
    # Merge history and fine-tune history for plotting
    acc = history.history['accuracy'] + history_finetune.history['accuracy']
    val_acc = history.history['val_accuracy'] + history_finetune.history['val_accuracy']
    loss = history.history['loss'] + history_finetune.history['loss']
    val_loss = history.history['val_loss'] + history_finetune.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history, history_finetune)


## 5.8 Validation and Testing<a id='5.8_Validation_Testing'></a>

In [ ]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_ds)
print(f'Test Accuracy: {test_acc}')

# Predictions
predictions = model.predict(test_ds)
predicted_classes = np.where(predictions > 0.5, 1, 0)

# Confusion Matrix
cm = confusion_matrix(test_ds.classes, predicted_classes)
print('Confusion Matrix')
print(cm)

# Classification Report
report = classification_report(test_ds.classes, predicted_classes, target_names=['Normal', 'Pneumonia'])
print('Classification Report')
print(report)

## 5.8.1 Plot Confusion Matrix<a id='5.7.1_Matrix'></a>

In [ ]:
def plot_confusion_matrix(cm, classes):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()

plt.figure(figsize=(6, 4))
plot_confusion_matrix(cm, classes=['Normal', 'Pneumonia'])
plt.show()

## 5.9 Save the Model<a id='5.9_Save_Model'></a>

In [ ]:
# Save the trained model
model.save('pneumonia_detection_model.h5')

## 5.10 Summary<a id='5.10_Summary'></a>

In this notebook, we implemented a deep learning model for pneumonia detection using EfficientNet-B7 pre-trained on ImageNet. The model was fine-tuned using techniques such as early stopping, learning rate reduction, and dropout to prevent overfitting.

We employed data augmentation on the training data to improve generalization. After training, the model achieved satisfactory results in terms of accuracy. The confusion matrix and classification report provided insights into the model's performance for each class (Normal vs Pneumonia). Further improvements could be achieved through techniques like MixUp augmentation, more extensive fine-tuning, or hyperparameter tuning.

